In [1]:

key ="2201050767bb417f928e06c7166304c6b41e485d"
b_key ="641DEDB0-361B-441D-BF2E-215378D97EB4"
from census import Census
from us import states
%matplotlib inline
import seaborn as sns


import matplotlib.pyplot as plt
import numpy as np
import requests
import pandas as pd
pd.options.mode.chained_assignment = None 
import os
import requests
import urllib.request, json 
import pybea


c = Census(key)


In [2]:


data = pybea.get_data(b_key,
                      DataSetName='RegionalIncome',
                      TableName='CA1',
                      LineCode=1,
                      GeoFips='COUNTY',
                      Year=[ '2016'],
                      ResultFormat='JSON')
data = pd.DataFrame(data)
data =data.drop_duplicates()
data = data.rename(index =str, columns = {"GeoFips":"geo_id","DataValue":"income_p_capita"})
data = data.drop(["CL_UNIT","Code","NoteRef","TimePeriod","UNIT_MULT"], axis = 1)



In [3]:
df = pd.read_excel("County_Rural_Lookup.xlsx")
df = df.drop("Unnamed: 8", axis =1)


In [4]:
df.columns = df.iloc[2]
df = df.drop([0, 1,2])
df = df.drop([3145, 3146,3147,3148,3149,3150])
df = df.rename(columns ={"2010 Census \nPercent Rural":"percent_rural"})
df = df.rename(columns = {"2015 GEOID": "geo_id"} )


In [5]:
rural = df[df.percent_rural >= 99.9]

len(rural)

704

In [6]:
m_rural = df[(df.percent_rural < 99.9) & (df.percent_rural > 49.9 )]
len(m_rural)

1185

In [7]:
urban = df[df.percent_rural <= 49.9]
len(urban)

1253

In [8]:
areas = pd.read_csv("final_county.csv")


In [9]:
areas = areas.drop(["Unnamed: 0" ,"B01001_003E","B01001_004E","B01001_005E","B01001_006E"
                   ,"B01001_027E","B01001_028E","B01001_029E","B01001_030E"],axis = 1)


In [10]:
areas.County = areas.County.astype(str)


In [11]:
areas['County'] = areas['County'].str.zfill(3)

In [12]:
areas.State = areas.State.astype(str)
areas['State'] = areas['State'].str.zfill(2)


In [13]:
areas["geo_id"] = areas.State + areas.County


In [14]:
unemployment = pd.read_excel("laucntycur14.xlsx")
unemployment.columns = unemployment.iloc[3]

unemployment = unemployment.drop([0, 1,2,3,4])

unemployment = unemployment.rename(index=str,columns ={"(%)":"unemployment_rate"})
unemployment["geo_id"] = unemployment["S_Code"] + unemployment["Code"]
unemployment.head()


3,LAUS Code,S_Code,Code,County Name/State Abbreviation,Period,Force,Employed,Unemployed,unemployment_rate,geo_id
5,CN0100100000000,01,001,"Autauga County, AL",Jun-17,26128,25015,1113,4.3,01001
6,CN0100300000000,01,003,"Baldwin County, AL",Jun-17,94176,90303,3873,4.1,01003
7,CN0100500000000,01,005,"Barbour County, AL",Jun-17,8328,7799,529,6.4,01005
8,CN0100700000000,01,007,"Bibb County, AL",Jun-17,8557,8139,418,4.9,01007
9,CN0100900000000,01,009,"Blount County, AL",Jun-17,24626,23533,1093,4.4,01009


In [15]:
x = rural.loc[:, ['geo_id']]
areas.head()

,Median Age,Median Age ME,Population,White,Black,Native American,Asian,Native Hawaiian,Other,Multiple,...,ninety,h_above_poverty,Median Household Income,acs_laborf,acs_unemployed,Name,County,State,under_18,geo_id
0,58.2,4.0,76,55.0,0.0,3.0,0.0,0.0,0.0,6.0,...,0.0,17.0,56875.0,39.0,0.0,"Loving County, Texas",301,48,6.0,48301
1,56.5,8.6,91,29.0,0.0,0.0,21.0,28.0,0.0,10.0,...,0.0,13.0,65625.0,76.0,0.0,"Kalawao County, Hawaii",005,15,1.0,15005
2,39.8,10.5,274,210.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,63.0,57083.0,171.0,7.0,"King County, Texas",269,48,56.0,48269
3,48.0,5.2,425,413.0,0.0,0.0,0.0,0.0,0.0,8.0,...,2.0,112.0,53750.0,213.0,0.0,"McPherson County, Nebraska",117,31,93.0,31117
4,41.0,6.0,437,426.0,0.0,0.0,0.0,0.0,1.0,10.0,...,4.0,99.0,40375.0,190.0,7.0,"Arthur County, Nebraska",005,31,138.0,31005


In [16]:
rural_result = pd.merge(x, areas, on='geo_id')

In [17]:
rural_result = pd.merge(rural_result,unemployment, on ="geo_id")

rural_result = rural_result[rural_result["Period"] == "Jun-18"]

len(rural_result)
rural_result.to_excel("final_rural.xlsx")
#df3 = df1[(~df1.col1.isin(common.col1))

In [18]:
data["geo_id"] = data["geo_id"].str.zfill(5)
rural_result = pd.merge(rural_result,data, on ="geo_id")
rural_result.to_excel("bea_final_rural.xlsx")

In [19]:
len(rural_result)

703

In [22]:
x = urban.loc[:, ['geo_id']]
urban_result = pd.merge(x, areas, on='geo_id')
len(urban_result)

1253

In [23]:
urban_result = pd.merge(urban_result,unemployment, on ="geo_id")


urban_result = urban_result[urban_result["Period"] == "Jun-18"]
len(urban_result)
urban_result.to_excel("final_urban.xlsx")

In [24]:
d = pd.read_excel("test_d_final.xlsx")
d["geo_id"] = d["geo_id"].astype(str)
d["geo_id"] = d["geo_id"].str.zfill(5)

test_urban = pd.merge(urban_result,d, on ="geo_id")
test_urban.to_excel("beafinal_urban.xlsx")
len(test_urban)

1239

In [25]:
x = m_rural.loc[:, ['geo_id']]
m_rural_result = pd.merge(x, areas, on='geo_id')


In [26]:
m_rural_result = pd.merge(m_rural_result,unemployment, on ="geo_id")

m_rural_result = m_rural_result[m_rural_result["Period"] == "Jun-18"]
m_rural_result.to_excel("final_mrural.xlsx")
testm_rural = pd.merge(m_rural, d, on ="geo_id")
testm_rural.to_excel("beafinal_mrural.xlsx")
y = pd.merge(m_rural_result, testm_rural, on ="geo_id", how = "outer",indicator = True)
y[y["_merge"] == "both"]

,geo_id,Median Age,Median Age ME,Population,White,Black,Native American,Asian,Native Hawaiian,Other,...,State_y,2015 Geography Name,Note,2010 Census Total Population,2010 Census Urban Population,2010 Census Rural Population,percent_rural,income_p_capita,GeoName,_merge
0,01005,38.7,0.6,26614,12212.0,12745.0,46.0,118.0,0.0,198.0,...,AL,"Barbour County, Alabama",NaN,27457,8844,18613,67.7896,825374.0,"Barbour, AL",both
1,01007,40.2,0.9,22572,16876.0,4788.0,81.0,14.0,0.0,0.0,...,AL,"Bibb County, Alabama",NaN,22915,7252,15663,68.3526,662619.0,"Bibb, AL",both
2,01009,40.8,0.4,57704,50582.0,899.0,171.0,92.0,0.0,130.0,...,AL,"Blount County, Alabama",NaN,57322,5760,51562,89.9515,1815973.0,"Blount, AL",both
3,01011,39.2,1.7,10552,2288.0,7967.0,84.0,84.0,0.0,1.0,...,AL,"Bullock County, Alabama",NaN,10914,5307,5607,51.3744,276257.0,"Bullock, AL",both
4,01013,40.6,0.4,20280,10704.0,8825.0,15.0,203.0,0.0,13.0,...,AL,"Butler County, Alabama",NaN,20947,6026,14921,71.2322,673803.0,"Butler, AL",both
5,01019,45.7,0.4,25897,23792.0,1193.0,152.0,86.0,0.0,0.0,...,AL,"Cherokee County, Alabama",NaN,25989,3707,22282,85.7363,850398.0,"Cherokee, AL",both
6,01021,38.7,0.7,43817,35321.0,4386.0,134.0,191.0,0.0,37.0,...,AL,"Chilton County, Alabama",NaN,43643,5785,37858,86.7447,1407904.0,"Chilton, AL",both
7,01025,42.0,0.9,24847,13213.0,11304.0,8.0,114.0,33.0,0.0,...,AL,"Clarke County, Alabama",NaN,25833,6205,19628,75.9803,807275.0,"Clarke, AL",both
8,01035,44.0,0.4,12697,6440.0,5868.0,19.0,14.0,0.0,1.0,...,AL,"Conecuh County, Alabama",NaN,13228,2520,10708,80.9495,376182.0,"Conecuh, AL",both
9,01039,42.9,0.4,37729,31509.0,4896.0,130.0,176.0,23.0,3.0,...,AL,"Covington County, Alabama",NaN,37765,11461,26304,69.6518,1211378.0,"Covington, AL",both
